In [3]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [4]:
project_list = ['bkromhout-realm-java','btraceio-btrace','bytedeco-javacpp',
                'codecentric-spring-boot-admin', 'codenvy-legacy-che-plugins', 'coobird-thumbnailator',
                'checkstyle-checkstyle_checkstyle'
       'cryptomator-cryptomator', 'dropwizard-dropwizard',
       'dropwizard-metrics', 'evant-gradle-retrolambda',
       'facebook-facebook-android-sdk',
       'facebook-facebook-java-business-sdk', 'facebook-fresco',
       'facebook-litho', 'facebook-react-native-fbsdk', 'google-cdep',
       'google-dagger', 'google-error-prone', 'google-gitiles',
       'google-openrtb', 'google-openrtb-doubleclick', 'grpc-grpc-java',
       'havarunner-havarunner', 'immutables-immutables',
       'ionic-team-capacitor', 'jankotek-mapdb',
       'javafunk-funk', 'javaparser-javaparser',
       'jboss-switchyard-release', 'jenkinsci-jenkins', 'jhy-jsoup',
       'joelittlejohn-jsonschema2pojo',
       'knowm-XChange', 'lettuce-io-lettuce-core', 'mockito-mockito',
       'mybatis-mybatis-3', 'naver-pinpoint', 'netty-netty', 'oblac-jodd',
       'openengsb-openengsb', 'oracle-oci-java-sdk', 'oracle-opengrok',
       'oracle-spacewalk-java',
       'permissions-dispatcher-PermissionsDispatcher', 'pxb1988-dex2jar',
       'quarkusio-quarkus', 'raphw-byte-buddy', 'real-logic-aeron',
       'redisson-redisson', 'rest-assured-rest-assured',
       'robolectric-robolectric', 'rzwitserloot-lombok',
       'scribejava-scribejava', 'seleniumHQ-selenium',
       'sofastack-sofa-rpc', 'spring-projects-spring-boot',
       'spring-projects-spring-security', 'springfox-springfox',
       'square-android-times-square', 'square-okhttp', 'square-retrofit',
       'swagger-api-swagger-core', 'testcontainers-testcontainers-java',
       'uber-NullAway', 'voxeolabs-moho', 'web3j-web3j']

print(len(project_list))

66


In [5]:
len_data_arr = []
results_data_arr_agglo = []
results_data_arr_bunch = []

bunch_bool = False
bunch_hypertune = ['turbomqincrw', 'turbomqincr', 'turbomqw', 'turbomq', 'basicmq']

for root, dirs, files in os.walk('Results_28112020'):
    for file in files:
        #print(file)
        #print(file.split('_')[-1])
        if file.split('_')[-1] == 'len.txt':
            print(file, 'len')
            proj_len_file = open('Results_28112020/' + file)
            proj_name = file.split('_')[0]
            proj_len = proj_len_file.readline().replace('\n','')
            len_data_arr.append([proj_name, proj_len])
            proj_len_file.close()
        elif file.split('_')[-1] == 'results.txt':
            #print(file, 'results')
            proj_result_file = open('Results_28112020/' + file)
            proj_name = file.split('_')[0]
            
            initial_line = 2 - 1
            initial_line_result = 5 - 1
            tmp_arr = []
            
            for i, line in enumerate(proj_result_file):
                
                if i == initial_line:
                    line = line.replace('MoJo', '')
                    line = line.replace('.rsf', '')
                    line = line.replace('(', '')
                    line = line.replace(')', '')
                    line = line.split('_')
                    name = line[1].split('/')[-1]
                    #version = line[0].split('-')[-1]
                    version = line[2]
                    
                    ## If gathers results for bunch
                    if line[3] == 'exhaustive' or line[3] == 'hillclimbing' or line[3] == 'ga':
                        
                        if line[4] in bunch_hypertune:
                            bunch_bool = True
                            method = line[3]
                            calculator = line[4]
                            tmp_arr.append(name)
                            tmp_arr.append(version)
                            tmp_arr.append(method)
                            tmp_arr.append(calculator)

                        initial_line += 5

                    
                    
                    ## Else gathers results for agglomerative
                    else:
                        agglo_bool = True
                        n_cluster = line[3]
                        affinity = line[4]
                        linkage = line[5]
                        #print(line)
                        #print('Version', version)
                        initial_line += 5
                        tmp_arr.append(name)
                        tmp_arr.append(version)
                        tmp_arr.append(n_cluster)
                        tmp_arr.append(affinity)
                        tmp_arr.append(linkage)
                        
                elif i == initial_line_result:
                    n_MoJo = line.split(' ')[-1].split('\n')[0]

                    #print(line)
                    initial_line_result += 5
                    tmp_arr.append(n_MoJo)
                    #tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
                    
                    if agglo_bool == True:
                        results_data_arr_agglo.append(tmp_arr)
                        agglo_bool = False
                    elif bunch_bool == True:
                        results_data_arr_bunch.append(tmp_arr)
                        bunch_bool = False
                    tmp_arr = []
                    
                    
                
                

aws-aws-sdk-java-v2_max_len.txt len
bdew-neiaddons_max_len.txt len
bkromhout-realm-java_max_len.txt len
btraceio-btrace_max_len.txt len
bytedeco-javacpp_max_len.txt len
checkstyle-checkstyle_max_len.txt len
codecentric-spring-boot-admin_max_len.txt len
codenvy-legacy-che-plugins_max_len.txt len
coobird-thumbnailator_max_len.txt len
crate-crate_max_len.txt len
cryptomator-cryptomator_max_len.txt len
dropwizard-dropwizard_max_len.txt len
dropwizard-metrics_max_len.txt len
evant-gradle-retrolambda_max_len.txt len
facebook-facebook-android-sdk_max_len.txt len
facebook-facebook-java-business-sdk_max_len.txt len
facebook-fresco_max_len.txt len
facebook-litho_max_len.txt len
facebook-react-native-fbsdk_max_len.txt len
flyway_flyway_max_len.txt len
google-cdep_max_len.txt len
google-dagger_max_len.txt len
google-error-prone_max_len.txt len
google-exoplayer_max_len.txt len
google-gitiles_max_len.txt len
google-openrtb-doubleclick_max_len.txt len
google-openrtb_max_len.txt len
grpc-grpc-java_max

In [6]:
pd.DataFrame(results_data_arr_agglo)

,0,1,2,3,4,5
0,aws-aws-sdk-java-v2,2.9.9,5,euclidean,complete,1280
1,aws-aws-sdk-java-v2,2.9.9,5,euclidean,average,1695
2,aws-aws-sdk-java-v2,2.9.9,5,euclidean,single,644
3,aws-aws-sdk-java-v2,2.9.9,5,l1,complete,1070
4,aws-aws-sdk-java-v2,2.9.9,5,l1,average,695
...,...,...,...,...,...,...
64693,web3j-web3j,v4.5.2,25,manhattan,average,66
64694,web3j-web3j,v4.5.2,25,manhattan,single,59
64695,web3j-web3j,v4.5.2,25,cosine,complete,205
64696,web3j-web3j,v4.5.2,25,cosine,average,251


In [7]:
pd.DataFrame(results_data_arr_bunch)

,0,1,2,3,4
0,aws-aws-sdk-java-v2,2.9.9,hillclimbing,turbomqincrw,1319
1,aws-aws-sdk-java-v2,2.9.9,hillclimbing,basicmq,1533
2,aws-aws-sdk-java-v2,2.9.9,hillclimbing,turbomqincr,1475
3,aws-aws-sdk-java-v2,2.9.9,hillclimbing,turbomq,1323
4,aws-aws-sdk-java-v2,2.9.9,hillclimbing,turbomqw,1541
...,...,...,...,...,...
11191,web3j-web3j,v4.5.2,ga,turbomqincrw,360
11192,web3j-web3j,v4.5.2,ga,basicmq,336
11193,web3j-web3j,v4.5.2,ga,turbomqincr,321
11194,web3j-web3j,v4.5.2,ga,turbomq,348


In [8]:
pd.DataFrame(len_data_arr)

,0,1
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154
...,...,...
74,swagger-api-swagger-core,763
75,testcontainers-testcontainers-java,151
76,uber-NullAway,106
77,voxeolabs-moho,639


In [9]:
agglo_results = pd.DataFrame(results_data_arr_agglo)
agglo_results.columns = ['project_name', 'project_version', 'cluster_division', 'affinity', 'linkage', 'MoJo_results']
agglo_results = agglo_results[agglo_results['project_name'].isin(project_list)]
agglo_results.head()

,project_name,project_version,cluster_division,affinity,linkage,MoJo_results
1440,bkromhout-realm-java,v0.89.0,5,euclidean,complete,91
1441,bkromhout-realm-java,v0.89.0,5,euclidean,average,147
1442,bkromhout-realm-java,v0.89.0,5,euclidean,single,87
1443,bkromhout-realm-java,v0.89.0,5,l1,complete,76
1444,bkromhout-realm-java,v0.89.0,5,l1,average,66


In [10]:
bunch_results = pd.DataFrame(results_data_arr_bunch)
bunch_results.columns = ['project_name', 'project_version', 'method', 'calculator', 'MoJo_results']
bunch_results = bunch_results[bunch_results['project_name'].isin(project_list)]
bunch_results.head()

,project_name,project_version,method,calculator,MoJo_results
300,bkromhout-realm-java,v0.89.0,hillclimbing,turbomqincrw,239
301,bkromhout-realm-java,v0.89.0,hillclimbing,basicmq,250
302,bkromhout-realm-java,v0.89.0,hillclimbing,turbomqincr,222
303,bkromhout-realm-java,v0.89.0,hillclimbing,turbomq,238
304,bkromhout-realm-java,v0.89.0,hillclimbing,turbomqw,246


In [11]:
bunch_results['project_name'].unique()
print(len(bunch_results['project_name'].unique()))

65


In [12]:
print(len(agglo_results['project_name'].unique()))

65


In [13]:
proj_len_df = pd.DataFrame(len_data_arr)
proj_len_df.columns = ['project_name', 'len']
proj_len_df.head()

,project_name,len
0,aws-aws-sdk-java-v2,2168
1,bdew-neiaddons,38
2,bkromhout-realm-java,352
3,btraceio-btrace,495
4,bytedeco-javacpp,154


In [14]:
## Remove flyway error
agglo_results = agglo_results[agglo_results['project_name'] != 'flyway']
agglo_results = agglo_results[agglo_results['project_name'] != 'mrniko-netty-socketio']



In [15]:
proj_len_df.to_csv('proj_len.csv', index=False)

In [16]:
ck_max = pd.read_csv('ck_max_194.csv')
ck_std = pd.read_csv('ck_std_194.csv')
ck_mean = pd.read_csv('ck_mean_194.csv')
ck_sum = pd.read_csv('ck_sum_194.csv')

In [17]:
ck_max_column = ['project_name', 'project_version']
ck_std_column = ['project_name', 'project_version']
ck_mean_column = ['project_name', 'project_version']
ck_sum_column = ['project_name', 'project_version']

excluded_columns = ['project_name', 'project_version']



for element in ck_max.columns:
    #print(element)
    if element not in excluded_columns:
        ck_max_column.append('feature_' + element + '_max')
   
        
        
for element in ck_std.columns:
    #print(element)
    if element not in excluded_columns:
        ck_std_column.append('feature_' + element + '_std')
        
for element in ck_mean.columns:
    #print(element)
    if element not in excluded_columns:
        ck_mean_column.append('feature_' + element + '_mean')
        
for element in ck_sum.columns:
    #print(element)
    if element not in excluded_columns:
        ck_sum_column.append('feature_' + element + '_sum')
    
        
ck_max.columns = ck_max_column
ck_std.columns = ck_std_column
ck_sum.columns = ck_sum_column
ck_mean.columns = ck_mean_column




In [62]:
ck_max['Instances'] = ck_max['project_name'] + '_' + ck_max['project_version']
ck_std['Instances'] = ck_std['project_name'] + '_' + ck_std['project_version']
ck_sum['Instances'] = ck_sum['project_name'] + '_' + ck_sum['project_version']
ck_mean['Instances'] = ck_mean['project_name'] + '_' + ck_mean['project_version']

ck_max = ck_max.drop(columns=['project_name', 'project_version'])
ck_std = ck_std.drop(columns=['project_name', 'project_version'])
ck_sum = ck_sum.drop(columns=['project_name', 'project_version'])
ck_mean = ck_mean.drop(columns=['project_name', 'project_version'])


In [63]:
main_df_agglo = agglo_results.merge(proj_len_df, on='project_name', how='left')
main_df_agglo['len'] = main_df_agglo['len'].astype('int32')
main_df_agglo['cluster_division'] = main_df_agglo['cluster_division'].astype('int32')
main_df_agglo['MoJo_results'] = main_df_agglo['MoJo_results'].astype('int32')
main_df_agglo['num_clusters'] = main_df_agglo['len'] // main_df_agglo['cluster_division']
main_df_agglo['accuracy'] = 1-  (main_df_agglo['MoJo_results'] / main_df_agglo['len'])
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l2']
main_df_agglo = main_df_agglo[main_df_agglo['affinity'] != 'l1']
main_df_agglo = main_df_agglo[main_df_agglo['accuracy'] != 1]
main_df_agglo = main_df_agglo.groupby(['project_name', 'project_version']).apply(lambda x: x.nlargest(1, "accuracy")).reset_index(drop=True)
main_df_agglo['Instances'] = main_df_agglo['project_name'] + '_' + main_df_agglo['project_version']
#main_df_agglo = main_df_agglo.drop(columns=['project_name', 'project_version'])

main_df_agglo = main_df_agglo.merge(ck_max, how='inner', on=['Instances'])
main_df_agglo = main_df_agglo.merge(ck_mean, how='inner', on=['Instances'])
main_df_agglo = main_df_agglo.merge(ck_std, how='inner', on=['Instances'])
main_df_agglo = main_df_agglo.merge(ck_sum, how='inner', on=['Instances'])

print(len(main_df_agglo))
main_df_agglo.head()

300


,project_name,project_version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,Instances,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,bkromhout-realm-java,v0.87.1,25,manhattan,average,27,352,14,0.923295,bkromhout-realm-java_v0.87.1,...,5004.0,4463.0,647.0,3854.0,196.0,452.0,46.0,0.0,15304.0,7499.0
1,bkromhout-realm-java,v0.87.2,25,manhattan,average,29,352,14,0.917614,bkromhout-realm-java_v0.87.2,...,5010.0,4476.0,647.0,3866.0,196.0,456.0,46.0,0.0,15315.0,7499.0
2,bkromhout-realm-java,v0.87.3,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.3,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
3,bkromhout-realm-java,v0.87.4,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.4,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
4,bkromhout-realm-java,v0.87.5,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.5,...,5032.0,4507.0,648.0,3897.0,196.0,462.0,46.0,0.0,15366.0,7499.0


In [64]:
main_df_agglo.groupby(['affinity']).agg(['count'])

,project_name,project_version,cluster_division,linkage,MoJo_results,len,num_clusters,accuracy,Instances,feature_cbo_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
affinity,,,,,,,,,,,,,,,,,,,,,
cosine,97,97,97,97,97,97,97,97,97,97,...,97,97,97,97,97,97,97,97,97,97
euclidean,74,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74
manhattan,129,129,129,129,129,129,129,129,129,129,...,129,129,129,129,129,129,129,129,129,129


In [65]:
main_df_agglo.groupby(['linkage']).agg(['count'])

,project_name,project_version,cluster_division,affinity,MoJo_results,len,num_clusters,accuracy,Instances,feature_cbo_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
linkage,,,,,,,,,,,,,,,,,,,,,
average,48,48,48,48,48,48,48,48,48,48,...,48,48,48,48,48,48,48,48,48,48
complete,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
single,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200


In [66]:
main_df_agglo.groupby(['cluster_division']).agg(['count'])

,project_name,project_version,affinity,linkage,MoJo_results,len,num_clusters,accuracy,Instances,feature_cbo_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
cluster_division,,,,,,,,,,,,,,,,,,,,,
5,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
7,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
10,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
15,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
20,24,24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24
25,227,227,227,227,227,227,227,227,227,227,...,227,227,227,227,227,227,227,227,227,227


In [67]:
main_df_bunch = bunch_results.merge(proj_len_df, on='project_name', how='left')
main_df_bunch['len'] = main_df_bunch['len'].astype('int32')
main_df_bunch['MoJo_results'] = main_df_bunch['MoJo_results'].astype('int32')
main_df_bunch['accuracy'] = 1-  (main_df_bunch['MoJo_results'] / main_df_bunch['len'])
main_df_bunch = main_df_bunch.groupby(['project_name', 'project_version']).apply(lambda x: x.nlargest(1, "accuracy")).reset_index(drop=True)
main_df_bunch['Instances'] = main_df_bunch['project_name'] + '_' + main_df_bunch['project_version']
#main_df_bunch = main_df_bunch.drop(columns=['project_name', 'project_version'])

main_df_bunch = main_df_bunch.merge(ck_max, how='inner', on=['Instances'])
main_df_bunch = main_df_bunch.merge(ck_mean, how='inner', on=['Instances'])
main_df_bunch = main_df_bunch.merge(ck_std, how='inner', on=['Instances'])
main_df_bunch = main_df_bunch.merge(ck_sum, how='inner', on=['Instances'])

print(len(main_df_bunch))
main_df_bunch.head()

300


,project_name,project_version,method,calculator,MoJo_results,len,accuracy,Instances,feature_cbo_max,feature_wmc_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,bkromhout-realm-java,v0.87.1,exhaustive,turbomqw,145,352,0.588068,bkromhout-realm-java_v0.87.1,72.0,315.0,...,5004.0,4463.0,647.0,3854.0,196.0,452.0,46.0,0.0,15304.0,7499.0
1,bkromhout-realm-java,v0.87.2,ga,turbomqincr,160,352,0.545455,bkromhout-realm-java_v0.87.2,72.0,315.0,...,5010.0,4476.0,647.0,3866.0,196.0,456.0,46.0,0.0,15315.0,7499.0
2,bkromhout-realm-java,v0.87.3,hillclimbing,turbomq,124,352,0.647727,bkromhout-realm-java_v0.87.3,74.0,315.0,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
3,bkromhout-realm-java,v0.87.4,exhaustive,turbomqw,162,352,0.539773,bkromhout-realm-java_v0.87.4,74.0,315.0,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
4,bkromhout-realm-java,v0.87.5,ga,turbomqincr,143,352,0.593750,bkromhout-realm-java_v0.87.5,74.0,315.0,...,5032.0,4507.0,648.0,3897.0,196.0,462.0,46.0,0.0,15366.0,7499.0


In [68]:
main_df_bunch.groupby(['method']).agg(['count'])

,project_name,project_version,calculator,MoJo_results,len,accuracy,Instances,feature_cbo_max,feature_wmc_max,feature_dit_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
method,,,,,,,,,,,,,,,,,,,,,
exhaustive,104,104,104,104,104,104,104,104,104,104,...,104,104,104,104,104,104,104,104,104,104
ga,81,81,81,81,81,81,81,81,81,81,...,81,81,81,81,81,81,81,81,81,81
hillclimbing,115,115,115,115,115,115,115,115,115,115,...,115,115,115,115,115,115,115,115,115,115


In [69]:
main_df_bunch.groupby(['calculator']).agg(['count'])

,project_name,project_version,method,MoJo_results,len,accuracy,Instances,feature_cbo_max,feature_wmc_max,feature_dit_max,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
calculator,,,,,,,,,,,,,,,,,,,,,
basicmq,62,62,62,62,62,62,62,62,62,62,...,62,62,62,62,62,62,62,62,62,62
turbomq,57,57,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
turbomqincr,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
turbomqincrw,73,73,73,73,73,73,73,73,73,73,...,73,73,73,73,73,73,73,73,73,73
turbomqw,59,59,59,59,59,59,59,59,59,59,...,59,59,59,59,59,59,59,59,59,59


In [70]:
agglo_minimal = main_df_agglo[['Instances', 'accuracy']]
agglo_minimal.columns = ['Instances', 'agglo_accuracy']
bunch_minimal = main_df_bunch[['Instances', 'accuracy']]
bunch_minimal.columns = ['Instances', 'bunch_accuracy']
combine_df = agglo_minimal.merge(bunch_minimal, on=['Instances'])

In [71]:
combine_df.sort_values(['agglo_accuracy'], ascending=False)

,Instances,agglo_accuracy,bunch_accuracy
100,facebook-facebook-java-business-sdk_v3.2.7,0.986577,0.479866
101,facebook-facebook-java-business-sdk_v3.2.8,0.969799,0.743289
102,facebook-facebook-java-business-sdk_v3.2.9,0.968121,0.736577
104,facebook-facebook-java-business-sdk_v3.3.1,0.966443,0.852349
108,facebook-facebook-java-business-sdk_v3.3.5,0.966443,0.842282
...,...,...,...
215,havarunner-havarunner_0.9.1,0.738095,0.476190
219,havarunner-havarunner_0.9.5,0.690476,0.571429
218,havarunner-havarunner_0.9.4,0.690476,0.595238
217,havarunner-havarunner_0.9.3,0.690476,0.547619


In [72]:
conditions = [
    (combine_df['agglo_accuracy'] > combine_df['bunch_accuracy']),
    (combine_df['bunch_accuracy'] >= combine_df['agglo_accuracy'])
]

values = ['agglo', 'bunch']

combine_df['algo_choice'] = np.select(conditions, values)

In [73]:
combine_df.sort_values(['agglo_accuracy'], ascending=False).head()

,Instances,agglo_accuracy,bunch_accuracy,algo_choice
100,facebook-facebook-java-business-sdk_v3.2.7,0.986577,0.479866,agglo
101,facebook-facebook-java-business-sdk_v3.2.8,0.969799,0.743289,agglo
102,facebook-facebook-java-business-sdk_v3.2.9,0.968121,0.736577,agglo
104,facebook-facebook-java-business-sdk_v3.3.1,0.966443,0.852349,agglo
108,facebook-facebook-java-business-sdk_v3.3.5,0.966443,0.842282,agglo


In [74]:
combine_df.groupby(['algo_choice']).agg(['count'])

,Instances,agglo_accuracy,bunch_accuracy
,count,count,count
algo_choice,,,
agglo,291,291,291
bunch,9,9,9


In [75]:
#combine_df.to_csv('combine_algo_choice_28112020.csv', index=False)

In [76]:
#combine_df.groupby('project_name').count().reset_index().sort_values(by='project_version')


In [77]:
print(len(combine_df['Instances'].unique()))
print(len(main_df_agglo['Instances'].unique()))
print(len(main_df_bunch['Instances'].unique()))

300
300
300


In [82]:
main_df_agglo.head()

,project_name,project_version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,Instances,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,bkromhout-realm-java,v0.87.1,25,manhattan,average,27,352,14,0.923295,bkromhout-realm-java_v0.87.1,...,5004.0,4463.0,647.0,3854.0,196.0,452.0,46.0,0.0,15304.0,7499.0
1,bkromhout-realm-java,v0.87.2,25,manhattan,average,29,352,14,0.917614,bkromhout-realm-java_v0.87.2,...,5010.0,4476.0,647.0,3866.0,196.0,456.0,46.0,0.0,15315.0,7499.0
2,bkromhout-realm-java,v0.87.3,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.3,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
3,bkromhout-realm-java,v0.87.4,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.4,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
4,bkromhout-realm-java,v0.87.5,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.5,...,5032.0,4507.0,648.0,3897.0,196.0,462.0,46.0,0.0,15366.0,7499.0


In [28]:
main_df_agglo.head()

,project_name,version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,cbo_max,...,numbersQty_sum,assignmentsQty_sum,mathOperationsQty_sum,variablesQty_sum,maxNestedBlocks_sum,anonymousClassesQty_sum,subClassesQty_sum,lambdasQty_sum,uniqueWordsQty_sum,modifiers_sum
0,aws-aws-sdk-java-v2,2.9.9,25,manhattan,single,281,2168,86,0.870387,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
1,aws-aws-sdk-java-v2,2.9.8,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
2,aws-aws-sdk-java-v2,2.9.7,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
3,aws-aws-sdk-java-v2,2.9.6,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0
4,aws-aws-sdk-java-v2,2.9.5,25,manhattan,single,282,2168,86,0.869926,138.0,...,5944.0,16957.0,2103.0,15768.0,1194.0,243.0,823.0,3656.0,139293.0,117054.0


In [85]:
g = main_df_agglo.groupby('project_name')

first = g['project_name', 'project_version'].head(1)
first.columns = ['project_name', 'firstRelease']

last = g['project_name', 'project_version'].tail(1)
last.columns = ['project_name', 'lastRelease']


project_first_last = first.merge(last, on='project_name')
#project_first_last = (pd.concat([g.head(1), g.tail(1)]).reset_index(drop=True).sort_values('project_name'))
pd.set_option('display.max_rows', 999)
pd.set_option('display.width', None)
project_first_last.head(71)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,project_name,firstRelease,lastRelease
0,bkromhout-realm-java,v0.87.1,v0.89.0
1,btraceio-btrace,v1.3.4,v1.3.9
2,bytedeco-javacpp,1.4,1.5.3
3,codecentric-spring-boot-admin,1.4.5,1.5.7
4,codenvy-legacy-che-plugins,3.13.4.4,3.9.5
5,coobird-thumbnailator,0.4.10,0.4.9
6,dropwizard-dropwizard,v2.0.11,v2.0.9
7,dropwizard-metrics,v4.1.10.1,v4.1.9
8,evant-gradle-retrolambda,v3.3.0-beta1,v3.7.0
9,facebook-facebook-android-sdk,sdk-version-5.15.1,sdk-version-5.9.0


In [48]:
len(project_first_last)

71

In [87]:
g_bunch = main_df_bunch.groupby('project_name')

first_bunch = g_bunch['project_name', 'project_version'].head(1)
first_bunch.columns = ['project_name', 'startRelease']

last_bunch = g_bunch['project_name', 'project_version'].tail(1)
last_bunch.columns = ['project_name', 'endRelease']


project_first_last_bunch = first_bunch.merge(last_bunch, on='project_name')
#project_first_last = (pd.concat([g.head(1), g.tail(1)]).reset_index(drop=True).sort_values('project_name'))
project_first_last_bunch.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,project_name,startRelease,endRelease
0,bkromhout-realm-java,v0.87.1,v0.89.0
1,btraceio-btrace,v1.3.4,v1.3.9
2,bytedeco-javacpp,1.4,1.5.3
3,codecentric-spring-boot-admin,1.4.5,1.5.7
4,codenvy-legacy-che-plugins,3.13.4.4,3.9.5
5,coobird-thumbnailator,0.4.10,0.4.9
6,dropwizard-dropwizard,v2.0.11,v2.0.9
7,dropwizard-metrics,v4.1.10.1,v4.1.9
8,evant-gradle-retrolambda,v3.3.0-beta1,v3.7.0
9,facebook-facebook-android-sdk,sdk-version-5.15.1,sdk-version-5.9.0


In [51]:
len(project_first_last_bunch)

74

In [52]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]
#print(Diff(li1, li2))


print(Diff(project_first_last_bunch['project_name'], project_first_last['project_name']))

['hibernate-hibernate-orm', 'realm-realm-java', 'mrniko-netty-socketio']


In [101]:
main_df_agglo.head()

,project_name,project_version,cluster_division,affinity,linkage,MoJo_results,len,num_clusters,accuracy,Instances,...,feature_numbersQty_sum,feature_assignmentsQty_sum,feature_mathOperationsQty_sum,feature_variablesQty_sum,feature_maxNestedBlocks_sum,feature_anonymousClassesQty_sum,feature_subClassesQty_sum,feature_lambdasQty_sum,feature_uniqueWordsQty_sum,feature_modifiers_sum
0,bkromhout-realm-java,v0.87.1,25,manhattan,average,27,352,14,0.923295,bkromhout-realm-java_v0.87.1,...,5004.0,4463.0,647.0,3854.0,196.0,452.0,46.0,0.0,15304.0,7499.0
1,bkromhout-realm-java,v0.87.2,25,manhattan,average,29,352,14,0.917614,bkromhout-realm-java_v0.87.2,...,5010.0,4476.0,647.0,3866.0,196.0,456.0,46.0,0.0,15315.0,7499.0
2,bkromhout-realm-java,v0.87.3,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.3,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
3,bkromhout-realm-java,v0.87.4,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.4,...,5028.0,4502.0,648.0,3892.0,196.0,459.0,46.0,0.0,15394.0,7500.0
4,bkromhout-realm-java,v0.87.5,25,manhattan,single,33,352,14,0.906250,bkromhout-realm-java_v0.87.5,...,5032.0,4507.0,648.0,3897.0,196.0,462.0,46.0,0.0,15366.0,7499.0


In [99]:
### Visualization
visual_bunch_df = main_df_bunch[['project_name', 'project_version', 'method', 'calculator', 'accuracy']].copy()
visual_bunch_df.columns = ['project_name', 'project_version', 'algorithm', 'calculator', 'MoJoFM']
visual_bunch_df['MoJoFM'] = visual_bunch_df['MoJoFM'] * 100
#visual_bunch_df.drop(columns=['MoJo_results'], inplace=True)
visual_bunch_df.sort_values(['MoJoFM'], ascending=False).head(10)

,project_name,project_version,algorithm,calculator,MoJoFM
187,google-openrtb,1.5.7,ga,turbomqincrw,90.909091
185,google-openrtb,1.5.5,hillclimbing,basicmq,90.909091
138,facebook-react-native-fbsdk,v0.10.2,exhaustive,turbomqincrw,90.476190
136,facebook-react-native-fbsdk,v0.10.0,hillclimbing,turbomq,90.476190
22,bytedeco-javacpp,1.4.2,hillclimbing,turbomqw,88.961039
189,google-openrtb,1.5.9,exhaustive,turbomqincr,88.636364
182,google-openrtb,1.5.2,exhaustive,turbomqincr,88.636364
184,google-openrtb,1.5.4,hillclimbing,basicmq,88.636364
186,google-openrtb,1.5.6,exhaustive,turbomq,88.636364
180,google-openrtb,1.5.11,hillclimbing,basicmq,86.363636


In [104]:
### Visualization
visual_agglo_df = main_df_agglo[['project_name', 'project_version', 'cluster_division', 'affinity', 'linkage',  'num_clusters', 'accuracy']].copy()
visual_agglo_df.columns = ['project_name', 'project_version', 'cluster_division', 'affinity', 'linkage',  'num_clusters', 'MoJoFM']
#visual_agglo_df.drop(columns=['MoJo_results'], inplace=True)
visual_agglo_df['MoJoFM'] = visual_agglo_df['MoJoFM'] * 100

visual_agglo_df.sort_values(['MoJoFM'], ascending=False).head(10)

,project_name,project_version,cluster_division,affinity,linkage,num_clusters,MoJoFM
100,facebook-facebook-java-business-sdk,v3.2.7,25,euclidean,single,23,98.657718
101,facebook-facebook-java-business-sdk,v3.2.8,25,euclidean,single,23,96.979866
102,facebook-facebook-java-business-sdk,v3.2.9,25,euclidean,single,23,96.812081
104,facebook-facebook-java-business-sdk,v3.3.1,25,cosine,single,23,96.644295
108,facebook-facebook-java-business-sdk,v3.3.5,25,cosine,single,23,96.644295
109,facebook-facebook-java-business-sdk,v3.3.6,25,cosine,single,23,96.644295
103,facebook-facebook-java-business-sdk,v3.3.0,25,cosine,single,23,96.476510
105,facebook-facebook-java-business-sdk,v3.3.2,25,euclidean,single,23,96.476510
106,facebook-facebook-java-business-sdk,v3.3.3,25,euclidean,single,23,96.476510
259,javafunk-funk,funk-0.2.0,25,cosine,single,10,96.268657
